In [1]:
pwd

'/Users/aidasaglinskas/Desktop/BC-FacExpr-1.2-fMRI-mainExp'

In [22]:
from matplotlib import pyplot as plt
from PIL import Image, ImageEnhance,ImageStat
import skvideo
import skvideo.io
import numpy as np
import os
from tqdm import tqdm
import alignfaces as afa
import cv2

In [23]:
def safe_mkdir(dir_name):
    if not os.path.exists(dir_name):
        os.mkdir(dir_name)

In [24]:
def load_video(fn):
    #fn = 'ID4_h1_1.mp4'
    videodata = skvideo.io.vread(fn)
    return videodata

In [25]:
def save_video(videodata,ofn='outvideo.mp4'):
    skvideo.io.vwrite(ofn, videodata.astype(np.uint8),outputdict={'-codec': 'h264', '-filter:v' : 'fps=29.97'})

In [26]:
files = [os.path.join('./stimuli/',file) for file in os.listdir('./stimuli/') if file.endswith('.mp4')]
files.sort()
files[0:5]

['./stimuli/id1d1.mp4',
 './stimuli/id1d2.mp4',
 './stimuli/id1f1.mp4',
 './stimuli/id1f2.mp4',
 './stimuli/id1h1.mp4']

In [7]:
def split_into_frames(vid_fn):
    videodata = load_video(vid_fn)
    nframes = videodata.shape[0]
    frame_filenames = []
    for f in range(nframes):
        frame = videodata[f,:,:,:]
        frame_ofn = vid_fn.replace('.mp4','').replace('./stimuli/','./stimuli_pics/')+f'frame-{f}.png'
        frame_filenames.append(frame_ofn)
        Image.fromarray(frame).save(frame_ofn)
    return frame_filenames,videodata

In [8]:
def get_video_frames(vid_fn):
    vid_name = vid_fn.split('/')[-1].replace('.mp4','')
    frames_bad = [i for i in os.listdir('./stimuli_pics/') if i.startswith(vid_name)]
    frames = [f'./stimuli_pics/{vid_name}frame-{i:01d}.png' for i in range(len(frames_bad))]
    return frames

In [9]:
def return_frame(image_fn,enlarge=0):
    
    trained_data = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    img = cv2.imread(image_fn)
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    face_cordinates = trained_data.detectMultiScale(gray_img)
    face_cordinates = face_cordinates + [-enlarge,-enlarge,enlarge*2,enlarge*2]
    
    return face_cordinates

In [10]:
 def crop_frame_to_rect(image_fn,rect,debug=False):
    img = cv2.imread(image_fn)
    img2 = cv2.imread(image_fn)
    for (x, y, w, h) in rect:
        cv2.rectangle(img, (x, y), (x+w, y+h), (0, 0, 0), -2)
        
    img2[img!=0]=0

    if debug==False:
        return img2
    else:
        return img2,img

In [11]:
def rect_size(mean_frame):
    for (x, y, w, h) in rect:
        size = w*h
    return size

In [12]:
def save_cropped(frame_filenames,videodata,vid_ofn):
    mask = np.array([crop_frame(frame_filename)[1] for frame_filename in frame_filenames])
    vid_mask = ((mask==0).sum(-1)==3).sum(axis=0)>1    
    videodata_cropped = videodata.copy()
    nframes = videodata_cropped.shape[0]
    for f in range(nframes):
        for c in range(3):
            videodata_cropped[f,:,:,c][~vid_mask]=0
            
    save_video(videodata_cropped,ofn=vid_ofn)

In [13]:
def resize_video(video_data,vid_size_target=(512,512)):
    from PIL import Image, ImageEnhance,ImageStat
    import numpy as np
    
    #vid_size_target = np.array((512,512))
    vid_size_target = np.array(vid_size_target)
    vid_size = np.array((video_data.shape[1],video_data.shape[2]))
    nframes = video_data.shape[0]
    nchannels = video_data.shape[-1]
    video_data_new = np.zeros((nframes,vid_size_target[0],vid_size_target[1],nchannels))
    for f in range(nframes):
        video_data_new[f,:,:,:] = np.array(Image.fromarray(video_data[f,:,:,:]).resize(vid_size_target,Image.ANTIALIAS))

    return video_data_new

In [14]:
def resize_video(video_data,vid_size_target=(512,512)):
    from PIL import Image, ImageEnhance,ImageStat
    import numpy as np
    
    #vid_size_target = np.array((512,512))
    vid_size_target = np.array(vid_size_target)
    vid_size = np.array((video_data.shape[1],video_data.shape[2]))
    nframes = video_data.shape[0]
    nchannels = video_data.shape[-1]
    video_data_new = np.zeros((nframes,vid_size_target[0],vid_size_target[1],nchannels))
    for f in range(nframes):
        video_data_new[f,:,:,:] = np.array(Image.fromarray(video_data[f,:,:,:]).resize(vid_size_target[-1::-1],Image.ANTIALIAS))
        
    return video_data_new

In [15]:
# plt.figure(figsize=(15,15))
# for i,file in tqdm(enumerate(files[0::10])):
#     vid_fn = file
#     frame_filenames,video_data = split_into_frames(vid_fn)

#     frame_rects = np.array([return_frame(frame_filename,enlarge=0)[0] for frame_filename in frame_filenames])
#     mean_frame = frame_rects.mean(axis=0).astype(int)

#     print(mean_frame)

#     c = [[mean_frame[0],mean_frame[1],256,256]]
#     print(c)

#     x = c[0][0] - int(mean_frame[3]*.1)
#     y = c[0][1] - int(mean_frame[3]*.15)
#     w = c[0][2] + int(mean_frame[3]*.4)
#     h = c[0][3] + int(mean_frame[3]*.4)


#     nframes = video_data.shape[0]
#     video_data_cropped = np.array([video_data[f,y:y+h,x:x+w,:] for f in range(nframes)])
#     #video_data_cropped = resize_video(video_data_cropped,vid_size_target=(384,256))
#     print(video_data_cropped.shape)
#     #save_video(video_data_cropped,vid_fn.replace('./stimuli/','./stimuli_cropped/'))

#     print(print(np.array((video_data_cropped.shape[1]/video_data_cropped.shape[2])).round(4)))
    
#     plt.subplot(4,4,i+1)
#     plt.imshow(video_data_cropped[0,:,:,:].astype(np.uint8))

#     #Image.fromarray(video_data_cropped[0,:,:,:].astype(np.uint8))

In [16]:
# for vid_fn in tqdm(files):
    
#     frame_filenames = get_video_frames(vid_fn)
#     video_data = load_video(vid_fn)
    
#     #frame_filenames,video_data = split_into_frames(vid_fn)
    
#     frame_rects = np.array([return_frame(frame_filename,enlarge=100)[0] for frame_filename in frame_filenames])
#     mean_frame = frame_rects.mean(axis=0).astype(int)

#     c = [[mean_frame[0],mean_frame[1],512,512]]

#     x = c[0][0]
#     y = c[0][1]
#     w = c[0][2]
#     h = c[0][3]

    
#     nframes = video_data.shape[0]
#     video_data_cropped = np.array([video_data[f,y:y+h,x:x+w,:] for f in range(nframes)])
#     video_data_cropped = resize_video(video_data_cropped,vid_size_target=(512,512))
#     save_video(video_data_cropped,vid_fn.replace('./stimuli/','./stimuli_cropped/'))

In [27]:
safe_mkdir('./stimuli_cropped')

for vid_fn in tqdm(files[50::]):
    
    frame_filenames = get_video_frames(vid_fn)
    video_data = load_video(vid_fn)
    
    #frame_filenames,video_data = split_into_frames(vid_fn)
    
    frame_rects = np.array([return_frame(frame_filename,enlarge=0)[0] for frame_filename in frame_filenames])
    mean_frame = frame_rects.mean(axis=0).astype(int)

    c = [[mean_frame[0],mean_frame[1],256,256]]

    x = c[0][0] - int(mean_frame[3]*.1)
    y = c[0][1] - int(mean_frame[3]*.15)
    w = c[0][2] + int(mean_frame[3]*.4)
    h = c[0][3] + int(mean_frame[3]*.4)
    
    nframes = video_data.shape[0]
    video_data_cropped = np.array([video_data[f,y:y+h,x:x+w,:] for f in range(nframes)])
    video_data_cropped = resize_video(video_data_cropped,vid_size_target=(256,256))
    save_video(video_data_cropped,vid_fn.replace('./stimuli/','./stimuli_cropped/'))

100%|███████████████████████████████████████████| 19/19 [01:15<00:00,  3.99s/it]
